In [3]:
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Search for any field in JSON object

In [4]:
def get_field_values(json_obj, field_name):
    """
    Returns a list of all values of the specified field in the JSON object
    with any level of nesting.
    """
    result = []
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            if key == field_name:
                result.append(value)
            else:
                result.extend(get_field_values(value, field_name))
    elif isinstance(json_obj, list):
        for item in json_obj:
            result.extend(get_field_values(item, field_name))
    return result

Expected ELO calculation

In [5]:
def expected_result(this, opponent):
    return 1 / (1 + 10 ** ((opponent - this) / 400))

This part scrapes the data into a df and provides intital ELO <b>---outdated---</b>

In [22]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

In [ ]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

all_players = []
matchcount = 0

for i in range(1, len(data)):
    participants = get_field_values(data[i], 'participants')
    if len(participants[0]) != 10:
        continue
    matchcount += 1
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        win = 1 if get_field_values(player, 'win')[0] else 0
        loss = 0 if get_field_values(player, 'win')[0] else 1
        
        new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

        # check if the player is already in the DataFrame
        if pid in playersdf['id'].tolist():
            # increment the number of matches played for the existing player
            playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
            playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
            playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
        else:
            # add the new player to the DataFrame
            playersdf = playersdf.append(new_player, ignore_index=True)
            
        
        
playersdf

Adds players to df and calculates player ELO

In [24]:
with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

all_players = []
matchcount = 0

for i in range(1, len(data)):
    participants = get_field_values(data[i], 'participants')
    if len(participants[0]) != 10:
        continue
    matchcount += 1
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        win = 1 if get_field_values(player, 'win')[0] else 0
        loss = 0 if get_field_values(player, 'win')[0] else 1
        
        new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

        # check if the player is already in the DataFrame
        if pid in playersdf['id'].tolist():
            # increment the number of matches played for the existing player
            playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
            playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
            playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
        else:
            # add the new player to the DataFrame
            playersdf = playersdf.append(new_player, ignore_index=True)
        
    winners = get_field_values(participants, 'win')
    if not any(winners):
        teamAELO = 0.0
        teamBELO = 0.0
        for n in range(5):
            teamAELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
        for n in range(5, 10):
            teamBELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
        expA = expected_result(teamAELO, teamBELO)
        deltaAELO = k * (0.5 - expA)
        expB = expected_result(teamBELO, teamAELO)
        deltaBELO = k * (0.5 - expB)
        print('-------------------------------------')
        print('\n', i, ' A ELO: ', teamAELO, ' B ELO: ', teamBELO)
        print(i, ' deltaAELO: ', deltaAELO, ' deltaBELO: ', deltaBELO)  
        
        for n in range(5):
            playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += deltaAELO
        for n in range(5, 10):
            playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += deltaBELO
        
        
    else:
        winnerteam = []
        loserteam = []
        winnerELO = 0.0
        loserELO = 0.0
        for n, w in enumerate(winners):
            if w:
                winnerteam.append(participants[0][n])
                winnerELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
            else:
                loserteam.append(participants[0][n])
                loserELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5

        k = 100.0
        expected_winner = expected_result(winnerELO, loserELO)
        delta_winnerELO = k * (1 - expected_winner)
        expected_loser = expected_result(loserELO, winnerELO)
        delta_loserELO = k * (0 - expected_loser)
        print('\n', i, ' WinnerELO: ', winnerELO, ' loserELO: ', loserELO)
        print(i, ' Delta_WinnerELO: ', delta_winnerELO, ' delta_loserELO: ', delta_loserELO)
    
        for n, w in enumerate(winners):
            if w:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_winnerELO
            else:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_loserELO
        

        
playersdf


 2  WinnerELO:  981.3707797938853  loserELO:  950.0
2  Delta_WinnerELO:  45.497610794540336  delta_loserELO:  -45.497610794540336

 3  WinnerELO:  990.4251842672484  loserELO:  937.7328900163947
3  Delta_WinnerELO:  42.47457827948783  delta_loserELO:  -42.47457827948783

 4  WinnerELO:  1026.131966640428  loserELO:  1082.18422011746
4  Delta_WinnerELO:  57.99730371265266  delta_loserELO:  -57.99730371265266

 5  WinnerELO:  1084.1292703530803  loserELO:  1024.1869164048076
5  Delta_WinnerELO:  41.458185721049915  delta_loserELO:  -41.45818572104991

 76  WinnerELO:  1125.5874560741302  loserELO:  1008.3905061946693
76  Delta_WinnerELO:  33.7458675125257  delta_loserELO:  -33.74586751252569

 78  WinnerELO:  974.6446386821436  loserELO:  942.5545122839553
78  Delta_WinnerELO:  45.39494720501666  delta_loserELO:  -45.39494720501667

 96  WinnerELO:  982.7287306837575  loserELO:  1026.8683905884257
96  Delta_WinnerELO:  56.31825155178413  delta_loserELO:  -56.31825155178413

 97  WinnerE


 504  WinnerELO:  1281.3695098798114  loserELO:  985.7788728606058
504  Delta_WinnerELO:  15.426208837090027  delta_loserELO:  -15.426208837090027

 505  WinnerELO:  1072.7519647247082  loserELO:  996.8970892355567
505  Delta_WinnerELO:  39.25381216497254  delta_loserELO:  -39.25381216497254

 506  WinnerELO:  1133.7715816694615  loserELO:  1045.5916347099064
506  Delta_WinnerELO:  37.57552359299234  delta_loserELO:  -37.57552359299235

 507  WinnerELO:  1112.0057768896804  loserELO:  932.9993722231868
507  Delta_WinnerELO:  26.299809322546785  delta_loserELO:  -26.29980932254678

 508  WinnerELO:  1146.50331011884  loserELO:  1008.0161111169143
508  Delta_WinnerELO:  31.062555529549364  delta_loserELO:  -31.062555529549368

 509  WinnerELO:  970.3526640235159  loserELO:  1030.0686499279968
509  Delta_WinnerELO:  58.510184542984526  delta_loserELO:  -58.510184542984526

 510  WinnerELO:  1554.5125699623181  loserELO:  1103.8492140747742
510  Delta_WinnerELO:  6.951089831913892  delta_


 568  WinnerELO:  1627.3567118229396  loserELO:  1201.4362466295138
568  Delta_WinnerELO:  7.930736255675475  delta_loserELO:  -7.930736255675479

 569  WinnerELO:  1091.0095974758701  loserELO:  1282.4907916101995
569  Delta_WinnerELO:  75.06822660316394  delta_loserELO:  -75.06822660316392

 570  WinnerELO:  1376.360630330496  loserELO:  1144.8984263085515
570  Delta_WinnerELO:  20.876206754669713  delta_loserELO:  -20.87620675466971

 571  WinnerELO:  1348.9705943917736  loserELO:  1397.2368370851657
571  Delta_WinnerELO:  56.90172851647903  delta_loserELO:  -56.90172851647903

 572  WinnerELO:  1316.6689675454754  loserELO:  1166.0778240790341
572  Delta_WinnerELO:  29.590552673139648  delta_loserELO:  -29.59055267313964

 573  WinnerELO:  1612.317433077305  loserELO:  1346.2595202186153
573  Delta_WinnerELO:  17.77666558922075  delta_loserELO:  -17.776665589220755

 574  WinnerELO:  1405.8723229082525  loserELO:  1340.3351085686863
574  Delta_WinnerELO:  40.67873103736771  delta_

id          ELO  \
0    eSENkVd2RZz7UPvDJV4DUGdEesrck79UoqufrU_zbnvzzU...   904.502389   
1    tjn0xgZp-NXT7nnqUkUyFij-WpkuyW0C48R-BueCtsJ96y...   904.502389   
2    cWegWX9cAlTTu89tJqlAVTx6UBaXwFvtlZ_vfec9BVCyxw...   904.502389   
3    4UXXNNnYVIw7-Yy1fUDX4g-HcXCWbg9seluDuuvZYyOjh5...   904.502389   
4    COFiGhpicYc5NBOQqt76w4byLgqIBo3IbpalTnl7fyaPTC...   904.502389   
..                                                 ...          ...   
384  7nciBasYULX3ZfR7zONSSmDjhqj9F1g3Br1ElxKpFJZIwq...   934.049903   
385  K7PZGV21WiS6YNYe6nXZ498ZiS9jKeun-C3J7X1npMbtpK...  1217.133410   
386  NzuXdkdnH86tpmVYluykXL4ywJ2QF-aT6EV4t_MhTSLcWF...  1217.133410   
387  5NdpwQH95lFFDULCBDI81zLbIOOE8V2iZHRJ0yE94vEWDF...  1170.920744   
388  _GkYFa_HqYZNYVtPa_fEbjUG2Gy3AGsagRi4gpT9P_r4qR...   826.824309   

     matches_played  wins  losses  
0               2.0   0.0     2.0  
1               2.0   0.0     2.0  
2               2.0   0.0     2.0  
3               2.0   0.0     2.0  
4               2.0   0.0     2.0  
..              ...   ...     ...  
384            10.0   4.0     6.0  
385            20.0  16.0     4.0  
386            20.0  16.0     4.0  
387            16.0  10.0     6.0  
388             6.0   0.0     6.0  

[389 rows x 5 columns]

Seeing how the ranking changes over many played rounds

In [7]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

In [8]:
plotdata = []


with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

for p in range(20):
    all_players = []

    for i in range(1, len(data)):
        participants = get_field_values(data[i], 'participants')
        if len(participants[0]) != 10:
            continue

        for player in participants[1]:
            pid = get_field_values(player, 'puuid')[0]
            win = 1 if get_field_values(player, 'win')[0] else 0
            loss = 0 if get_field_values(player, 'win')[0] else 1

            new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

            # check if the player is already in the DataFrame
            if pid in playersdf['id'].tolist():
                # increment the number of matches played for the existing player
                playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
                playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
                playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
            else:
                # add the new player to the DataFrame
                playersdf = playersdf.append(new_player, ignore_index=True)

        winners = get_field_values(participants, 'win')
        if not any(winners):
            #TODO draw needs to be handled
            continue

        winnerteam = []
        loserteam = []
        winnerELO = 0.0
        loserELO = 0.0
        for n, w in enumerate(winners):
            if w:
                winnerteam.append(participants[0][n])
                winnerELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
            else:
                loserteam.append(participants[0][n])
                loserELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5

        k = 32.0
        expected_winner = expected_result(winnerELO, loserELO)
        delta_winnerELO = k * (1 - expected_winner)
        expected_loser = expected_result(loserELO, winnerELO)
        delta_loserELO = k * (0 - expected_loser)

        for n, w in enumerate(winners):
            if w:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_winnerELO
            else:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_loserELO
    
    plotdata.append(playersdf.loc[208, 'ELO'])

In [19]:
playersdf = playersdf.sort_values(by = 'ELO', ascending = False)

In [20]:
playersdf[['id', 'ELO', 'matches_played']].to_json('import.json', orient='records')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(20), plotdata)

In [ ]:
exp = pd.DataFrame({'x':range(20), 'y':plotdata})
exp.to_csv('exp.csv')

In [ ]:
with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

participants = get_field_values(data[2], 'participants')
player = participants[1][0]
pid = get_field_values(player, 'puuid')[0]
playerstats = pd.DataFrame(player)
playerstats.to_csv('playerdata.csv')
playerstats.drop('styles', inplace = True)
playerstats.drop('perks', axis = 1, inplace = True)
playerstats

In [ ]:
for i in range(1, 5):
    participants = get_field_values(data[i], 'participants')
    if len(participants[0]) != 10:
        continue
    
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        playerstats = pd.DataFrame(player)
        playerstats.head()
        
    player = participants[1][0]
    pid = get_field_values(player, 'puuid')[0]
    playerstats = pd.DataFrame(player)
    playerstats.head()